In [ ]:
%matplotlib qt
from nf_realtime import NF_Realtime
import mne
import mne_lsl
import bsl
import time

In [ ]:
nf = NF_Realtime(freq_range=(8, 13),
                artifact_rejection=False,
                return_delay=True,
                params=None,
                verbose=False)


nf.connect(connection_params=None, mock_lsl=True)

time.sleep(5)
nf.record_baseline(baseline_duration=10)

In [ ]:
time.sleep(2)
acq_delays, method_delays = nf.record_main(duration=60, modality="source_graph")

In [4]:
import numpy as np
fname = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1/subject_539_alpha_aparc_mean.npy'
stc_in_labels = np.load(file=fname, allow_pickle=True)

In [ ]:
def log_degree_barrier(X, dist_type, alpha, beta, step, w0, maxit, rtol):


        # Parse X
        N = X.shape[0]
        z = pdist(X, dist_type)  # Pairwise distances
        z = z/np.max(z) ###  ADDITION TO ORIGINAL CODE

        # Parse stepsize
        if (step <= 0) or (step > 1):
                raise ValueError("step must be a number between 0 and 1.")

        # Parse initial weights
        w0 = np.zeros(z.shape) if w0 is None else w0
        if (w0.shape != z.shape):
                raise ValueError("w0 must be of dimension N(N-1)/2.")

        # Get primal-dual linear map
        K, Kt = weight2degmap(N)
        norm_K = np.sqrt(2 * (N - 1))

        # Assemble functions in the objective
        f1 = functions.func()
        f1._eval = lambda w: 2 * np.dot(w, z)
        f1._prox = lambda w, gamma: np.maximum(0, w - (2 * gamma * z))

        f2 = functions.func()
        f2._eval = lambda w: - alpha * np.sum(np.log(np.maximum(
        np.finfo(np.float64).eps, K(w))))
        f2._prox = lambda d, gamma: np.maximum(
        0, 0.5 * (d + np.sqrt(d**2 + (4 * alpha * gamma))))

        f3 = functions.func()
        f3._eval = lambda w: beta * np.sum(w**2)
        f3._grad = lambda w: 2 * beta * w
        lipg = 2 * beta

        # Rescale stepsize
        stepsize = step / (1 + lipg + norm_K)

        # Solve problem
        solver = solvers.mlfbf(L=K, Lt=Kt, step=stepsize)
        problem = solvers.solve([f1, f2, f3], x0=w0, solver=solver, maxit=maxit,
                                rtol=rtol, verbosity=None)

        # Transform weight matrix from vector form to matrix form
        graph_matrix = squareform(problem['sol'])

        return graph_matrix